In [1]:
# -*- coding: utf-8 -*-
from urllib2 import urlopen
import urllib2
from bs4 import BeautifulSoup
import io
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import numpy as np
import re
import json
import time

# Part 1

In [365]:
# table raws
class BasicRaw:
    def __init__(self):
        self.text = "None"
    def get_text(self):
        return self.text
class TitleRaw(BasicRaw):
    def __init__(self, title):
        self.text = title

class KeyValueRaw(BasicRaw):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.text = key+" "+value
    def get_kvp(self):
        return self.key, self.value
class FunctionsAndOther(BasicRaw):
    def __init__(self, string, prev_header="None"):
        self.text = string
        self.prev_header = prev_header
class Description(BasicRaw):
    def __init__(self, text):
        self.text = text
    def __str__(self):
        return self.text
class Raw:
    # takes raw raw basically from tbody tag:) and that return one of TitleRow or KeyValueRaw object
    # prev is previous title
    def __init__(self, soup, prev="None"):
        self.soup = soup
        self.prev = prev
    def get_particular_raw_data(self):
        req = self.soup.find('span', id="watchNotes")
        if not(req is None):
            return Description(req.text)
        req = self.soup.find('h4')
        if not(req is None):
            # find a header raw
            return TitleRaw(req.text)
        req = self.soup.find_all('td')
        if len(req) == 2:
            # found key-value pair (we hope)
            return KeyValueRaw(req[0].text, req[1].text)
        else:
            return FunctionsAndOther(','.join([i.text for i in req]), self.prev)

In [371]:
class Page:
    def __init__(self, url, proxy_seed = 0, proxies=None):
        """
            this class scrap data form web page that include all info about item
            proxies - array of proxy ips
            proxy_seed is variable that determine proxy_id - can be the counter of Page class usage
        """
        self.url = url
        if proxies is None:
            self.proxies = []
        else:
            self.proxies = proxies
        self.ip_usage = proxy_seed
        # get's beautiful soup of page
        self.set_soup()
        # all data form rows
        self.rows = []
    def get_page(self):
        # open new ip each time using counter ip_usage
        prx = self.ip_usage % (len(self.proxies) + 1)
        self.ip_usage = (self.ip_usage + 1) % (len(self.proxies) + 1)
        if (prx == 0):
            print("usual ip")
            test_proxy = urllib2.ProxyHandler({})
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            return urllib2.urlopen(self.url)
        else:
            prx -= 1
            print self.proxies[prx]
            test_proxy = urllib2.ProxyHandler(self.proxies[prx])
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            answer = urllib2.urlopen(self.url)
            return answer
    def set_soup(self):
        page = self.get_page()
        soup = BeautifulSoup(page, 'html.parser')
        self.soup = soup
    def get_table(self):
        # put all data from tables in array of raws 
        # where each raw is a particular object selected in Raw class
        data = []
        for table in self.soup.find_all('tbody'):
            # this is title object /is a inner rows-sub-box title/
            prev = "None"
            for raw in table.find_all('tr'):
                print raw.prettify()
                r = Raw(raw, prev)
                partic_obj = r.get_particular_raw_data()
                if isinstance(partic_obj, TitleRaw):
                    prev = partic_obj
                print partic_obj
                print "/////////"
                data.append(partic_obj)
        return data
    def get_js_data(self):
        data = self.soup.find_all('script')
        var_names = ["watchId", "merchantId", "watchName", "originalPath", "uuid", "merchantPage"]
        data_dict = {}
        for var_name in var_names:
            for i in data:
                values = re.findall(r'var {} = "(.*?)";'.format(var_name), str(i), re.DOTALL | re.MULTILINE)
                if len(values) != 0:
                    data_dict[var_name] = values[0]
        return data_dict
    def get_from_table(self):
        # get pandas object containing all data from raws
        raws = self.get_table()
        data = self.get_js_data()
        for raw in raws:
            if isinstance(raw, KeyValueRaw):
                r_kvp = raw.get_kvp()
                if r_kvp[0] in data:
                    print data
                    # if keys collided
                    continue
                data[r_kvp[0]] = r_kvp[1]
            elif isinstance(raw, Description):
                data["_Description_"] = str(raw)
            elif isinstance(raw, FunctionsAndOther):
                if "_Functions_and_other" in data:
                    data["_Functions_and_other"] += '; ' + raw.get_text()
                else:
                    data["_Functions_and_other"] = raw.get_text()
        return data
    

In [372]:
test_item_url = "http://www.chrono24.com/gevril/avenue-of-americas-chrono-limited-edition--id3526426.htm?SETLANG=en_US&SETCURR=EUR"

In [373]:
page = Page(test_item_url)

usual ip


In [374]:
page.get_table()

<tr>
 <td colspan="2">
  <h4 class="h4-lead text-left m-b-0">
   Basic Info
  </h4>
 </td>
</tr>

<__main__.TitleRaw instance at 0x10c5c1170>
/////////
<tr>
 <td>
  <strong>
   Brand
  </strong>
 </td>
 <td>
  <a class="link-inverted" href="/gevril/index.htm" title="Gevril">
   Gevril
  </a>
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10c5c1638>
/////////
<tr>
 <td>
  <strong>
   Ref. No.
  </strong>
 </td>
 <td>
  5015
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10d7069e0>
/////////
<tr>
 <td>
  <strong>
   Code
  </strong>
 </td>
 <td>
  MEMOASIM
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10d6723b0>
/////////
<tr>
 <td>
  <strong>
   Movement
  </strong>
 </td>
 <td>
  Automatic
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10d6723f8>
/////////
<tr>
 <td>
  <strong>
   Case material
  </strong>
 </td>
 <td>
  Steel
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10d672440>
/////////
<tr>
 <td>
  <strong>
   Bracelet material
  </strong>
 </td>
 <td>
  Steel
 

[<__main__.TitleRaw instance at 0x10c5c1170>,
 <__main__.KeyValueRaw instance at 0x10c5c1638>,
 <__main__.KeyValueRaw instance at 0x10d7069e0>,
 <__main__.KeyValueRaw instance at 0x10d6723b0>,
 <__main__.KeyValueRaw instance at 0x10d6723f8>,
 <__main__.KeyValueRaw instance at 0x10d672440>,
 <__main__.KeyValueRaw instance at 0x10d672488>,
 <__main__.KeyValueRaw instance at 0x10d6724d0>,
 <__main__.KeyValueRaw instance at 0x10d672518>,
 <__main__.KeyValueRaw instance at 0x10d6725f0>,
 <__main__.KeyValueRaw instance at 0x10d672680>,
 <__main__.KeyValueRaw instance at 0x10d672638>,
 <__main__.TitleRaw instance at 0x10d6726c8>,
 <__main__.KeyValueRaw instance at 0x10d672710>,
 <__main__.TitleRaw instance at 0x10d6727a0>,
 <__main__.KeyValueRaw instance at 0x10d6727e8>,
 <__main__.KeyValueRaw instance at 0x10d672878>,
 <__main__.KeyValueRaw instance at 0x10d6728c0>,
 <__main__.KeyValueRaw instance at 0x10d672908>,
 <__main__.KeyValueRaw instance at 0x10d672998>,
 <__main__.TitleRaw instance 

In [376]:
page.get_from_table()

<tr>
 <td colspan="2">
  <h4 class="h4-lead text-left m-b-0">
   Basic Info
  </h4>
 </td>
</tr>

<__main__.TitleRaw instance at 0x10e7048c0>
/////////
<tr>
 <td>
  <strong>
   Brand
  </strong>
 </td>
 <td>
  <a class="link-inverted" href="/gevril/index.htm" title="Gevril">
   Gevril
  </a>
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10e704758>
/////////
<tr>
 <td>
  <strong>
   Ref. No.
  </strong>
 </td>
 <td>
  5015
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10e704830>
/////////
<tr>
 <td>
  <strong>
   Code
  </strong>
 </td>
 <td>
  MEMOASIM
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10e704908>
/////////
<tr>
 <td>
  <strong>
   Movement
  </strong>
 </td>
 <td>
  Automatic
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10e704950>
/////////
<tr>
 <td>
  <strong>
   Case material
  </strong>
 </td>
 <td>
  Steel
 </td>
</tr>

<__main__.KeyValueRaw instance at 0x10e704998>
/////////
<tr>
 <td>
  <strong>
   Bracelet material
  </strong>
 </td>
 <td>
  Steel
 

{u'Availability': u'Available now',
 u'Bracelet color': u'Steel',
 u'Bracelet material': u'Steel',
 u'Brand': u'Gevril',
 u'Case diameter': u'34 x 44 mm',
 u'Case material': u'Steel',
 u'Clasp': u'\n                     Fold clasp, hidden',
 u'Code': u'MEMOASIM',
 u'Dial': u'Silver',
 u'Dial numerals': u'Arabic numerals',
 u'Gender': u"Men's watch/Unisex",
 u'Glass': u'Sapphire Glass',
 u'Location': u'United States, New York, Hartsdale',
 u'Movement': u'Automatic',
 u'Price': u'\n                  price on request',
 u'Ref. No.': u'5015',
 '_Description_': 'Gents limited edition Gevril Avenue of Americas Chrono in stainless steel, Ref#5015. No.38 of a limited production of 500. On stainless steel bracelet with concealed double folding clasp, sapphire caseback, 34mm x 44mm case dimensions. Self winding movement, chronograph functions. Silver guilloche dial with sub dials for 30 minutes, 12 hours and seconds; applied rose Arabic numerals. Winding box. Excellent condition.',
 '_Functions_

In [377]:
page.get_js_data()

{'merchantId': '3144',
 'originalPath': '/gevril/avenue-of-americas-chrono-limited-edition--id3526426.htm',
 'uuid': '0ee50f7c-cf38-43a0-8d9a-f3304fe9e598',
 'watchId': '3526426',
 'watchName': 'Gevril Avenue of Americas Chrono Limited Edition'}

# Part 2

In [378]:
MainData = dict()

In [379]:
main_page = "http://www.chrono24.com/chronoswiss/index.htm"
# pages ase ..index, index-2, index-3, index-4

In [380]:
txt = urllib2.urlopen(main_page).read()

In [381]:
data = BeautifulSoup(txt, 'html.parser')

In [382]:
hrefs = data.find_all("div", class_="item")

In [383]:
urls = []
for h in hrefs:
    url_add = h.find("a")["href"]
    urls.append(main_page + url_add)

In [384]:
for i in urls:
    page_ = Page(i)
    print page_.get_from_table()

usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
usual ip
{}
